In [ ]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)
warnings.filterwarnings("ignore", message=r"cannot", category=UserWarning)

In [ ]:
import os
import sys
sys.path.append('..')

from carla.recourse_methods import Dice, Wachter
from copy import deepcopy
from model_shifts import (DynamicCsvCatalog,
                          RecourseExperiment,
                          RecourseGenerator,
                          train_model)

os.environ["CUDA_VISIBLE_DEVICES"] = ""

#     torch.manual_seed(0)
#     random.seed(0)
#     np.random.seed(0)

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [3]:
# Load the dataset from a csv file
dataset = DynamicCsvCatalog(file_path="../datasets/unimodal_dataset_2.csv", 
                            continuous=['feature1', 'feature2'], categorical=[],
                            immutables=[], target='target', test_size=0.3)

[INFO] Balance: train set 0.5, test set 0.5 [dynamic_csv_catalog.py __init__]


In [4]:
hyper_params = {'learning_rate': 0.04, 'epochs': 2, 'batch_size': 1, 'hidden_size': [4]}

# Recourse generated by DICE is compared with the Wachter generator, as they may modify data differently
# we need to keep track of two models and two datasets and update them independently
model = train_model(dataset, hyper_params)
generators = [
              RecourseGenerator('DICE_1', deepcopy(dataset), deepcopy(model),
                                Dice, {"num": 1}, hyper_params, 60),
            #   RecourseGenerator('DICE_5', deepcopy(dataset), deepcopy(model),
            #                     Dice, {"num": 5}, hyper_params, 120),
              # RecourseGenerator('Wachter', deepcopy(dataset), deepcopy(model),
              #                   Wachter, {"loss_type": "BCE", "t_max_min": 2}, hyper_params, 120)
             ]

# SET NAME BEFORE THE EXPERIMENT
name='test'
experiment = RecourseExperiment(dataset, model, generators, experiment_name=name)
experiment.run(total_recourse=0.80, recourse_per_epoch=0.05)
experiment.save_data()

[INFO] Training the initial model [recourse_generator.py train_model]
[INFO] Updating the Wachter model [recourse_generator.py update_model]
[INFO] Retraining the model [recourse_generator.py train_model]
[INFO] Current balance: train set 0.5, test set 0.5 [dynamic_mlmodel_catalog.py retrain]
[INFO] Starting epoch 1 [recourse_experiment.py run]
[INFO] Applying the Wachter generator. [recourse_generator.py apply_recourse_with_timeout]
[INFO] Generating counterfactual 1 with Wachter [recourse_generator.py apply_recourse_with_timeout]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Generating counterfactual 2 with Wachter [recourse_generator.py apply_recourse_with_timeout]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Generating counterfactual 3 with Wachter [recourse_generator.py apply_recourse_with_timeout]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Updating the Wachter model [recourse_generato